# 1- Packages Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 2- One cell to run everything

In [2]:

# Transfer all the notebook into once cell to run everything for different timelag at the same time

# # 1- Packages Import

# In[178]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# # 2- Approach

# Here, we analyze the different targets for our predictive problem.


# We don't go here into great depth about the interest of each target variable.

# # 3- Data import

# We import here our preprocessed masterfiles.




#for timelag in ['2','4','6','24']:
#in the endwe chose to focus on 2h
for timelag in ['2']:
    # In[180]:

    data = pd.read_csv('../data/Creatinine_data/master_file_different_hours/Preprocessed_Masterfile{}h.csv'.format(timelag), index_col=0)


    # In[181]:

    #print(data.shape)
    #data.head(5)


    # # 4- Creatinine filter

    # In[182]:

    th_cr = 1.2
    target_baseline = 'Cr_value_baseline'
    target_initial = 'Cr_value_initial'



    data_filter = data.loc[data[target_initial]>=th_cr]


    #print(data.shape)
    #print(data_filter.shape)


    # # 5- Targets

    # ### 5.1- Split X-y


    col_target = 'Cr_value_baseline'
    new_col_target = 'target_baseline'



    X = data.drop(col_target, axis=1)
    y = data[[col_target]]
    y = y.rename(columns={col_target: new_col_target})


    #print(X.shape)
    #print(y.shape)


    X_filter = data_filter.drop(col_target, axis=1)
    y_filter = data_filter[[col_target]]
    y_filter = y_filter.rename(columns={col_target: new_col_target})


    #print(X_filter.shape)
    #print(y_filter.shape)
    
    y['baseline_initial']=data['Cr_value_initial']
    y_filter['baseline_initial']=data_filter['Cr_value_initial']


    # ### 5.2- Targets preparation

    # ##### Drop of Cr


    y['target_drop'] = y[new_col_target] - data[target_initial]
    y_filter['target_drop'] = y_filter[new_col_target] - data_filter[target_initial]


    # ##### Drop in %


    def create_col_and_baseline(y,y_filter,funct,name):
        y['target_'+name]=y[new_col_target].apply(funct)
        y['target_'+name+'_baseline']=y['baseline_initial'].apply(funct)
        y_filter['target_'+name]=y_filter[new_col_target].apply(funct)
        y_filter['target_'+name+'_baseline']=y_filter['baseline_initial'].apply(funct)



    y['target_drop_percentage'] = y['target_drop']/y[new_col_target]
    y_filter['target_drop_percentage'] = y_filter['target_drop']/y_filter[new_col_target]


    # ##### Drop in % binary



    def is_low_percentage(chf):
        return abs(chf) <0.15
    
    def AKI(chf):
        return chf < -0.5
    
    def baseline_AKI(chf):
        return 0
        #return chf < 3



    y['target_drop_percentage_bin'] = y['target_drop_percentage'].apply(is_low_percentage).astype(int)
    y_filter['target_drop_percentage_bin'] = y_filter['target_drop_percentage'].apply(is_low_percentage).astype(int)
    
    y['target_AKI'] = y['target_drop_percentage'].apply(AKI).astype(int)
    y_filter['target_AKI'] = y_filter['target_drop_percentage'].apply(AKI).astype(int)
    
    y['target_AKI_baseline'] = y['baseline_initial'].apply(baseline_AKI).astype(int)
    y_filter['target_AKI_baseline'] = y_filter['baseline_initial'].apply(baseline_AKI).astype(int)
    
    
    

    # ##### Drop in % category


    
    def is_low_percentage_cate(chf):
        if abs(chf) <0.10:
            return 0
        elif 0.10<=abs(chf) <0.20:
            return 1
        else:
            return 2

    create_col_and_baseline(y,y_filter,is_low_percentage_cate,'drop_percentage_cate')
    

    # ##### Binary creatinine

    def is_low(chf):
        return int(chf < 1.2)


    create_col_and_baseline(y,y_filter,is_low,'bin')
    


    # ##### 3 categories creatinine


    def Cr_cate(chf):
        if chf <1:
            return 0
        elif 1<= chf <1.2:
            return 1
        elif 1.2 <= chf:
            return 2


    create_col_and_baseline(y,y_filter,Cr_cate,'cat')

   

    # ### 5.3- Targets overview

    print(y.shape)
    #y.head()



    print(y_filter.shape)
    #y_filter.head()
    y['target_baseline_baseline']=y['baseline_initial']
    y_filter['target_baseline_baseline']=y_filter['baseline_initial']
    
    cols_in_order=['baseline_initial', 'target_baseline', 'target_baseline_baseline', 'target_bin',
       'target_bin_baseline','target_AKI',
       'target_AKI_baseline', 'target_drop',
       'target_drop_percentage', 'target_drop_percentage_bin','target_drop_percentage_cate',
       'target_drop_percentage_cate_baseline',  'target_cat', 'target_cat_baseline']

    y=y[cols_in_order]
    y_filter=y_filter[cols_in_order]


    # # 6- Export


    export_name_X = '../data/Creatinine_data/X_not_imputed_{}h.csv'.format(timelag)
    X.to_csv(export_name_X)

    export_name_X_filter = '../data/Creatinine_data/X_filter_not_imputed_{}h.csv'.format(timelag)
    X_filter.to_csv(export_name_X_filter)



    export_name_y = '../data/Creatinine_data/y_targets_{}h.csv'.format(timelag)
    y.to_csv(export_name_y)

    export_name_y_filter = '../data/Creatinine_data/y_targets_filter_{}h.csv'.format(timelag)
    y_filter.to_csv(export_name_y_filter)
    






(4643, 13)
(1719, 13)


In [14]:
y.columns

Index(['baseline_initial', 'target_baseline', 'target_baseline_baseline',
       'target_bin', 'target_bin_baseline', 'target_AKI',
       'target_AKI_baseline', 'target_drop', 'target_drop_percentage',
       'target_drop_percentage_bin', 'target_drop_percentage_cate',
       'target_drop_percentage_cate_baseline', 'target_cat',
       'target_cat_baseline'],
      dtype='object')